In [1]:
import pandas as pd
import sklearn.metrics
import sklearn.linear_model
from sklearn.linear_model import LinearRegression
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
import sklearn.tree
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.metrics import mean_squared_error
import sklearn.decomposition
import imblearn.over_sampling
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv("bmw.csv")                         
df.head()

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,price,sold_at
0,BMW,118,140411,100,01/02/2012,diesel,black,convertible,11300,01/01/2018
1,BMW,M4,13929,317,01/04/2016,petrol,grey,convertible,69700,01/02/2018
2,BMW,320,183297,120,01/04/2012,diesel,white,convertible,10200,01/02/2018
3,BMW,420,128035,135,01/07/2014,diesel,red,convertible,25100,01/02/2018
4,BMW,425,97097,160,01/12/2014,diesel,silver,convertible,33400,01/04/2018


In [3]:
del df['maker_key']
del df['model_key']
df.head()

,mileage,engine_power,registration_date,fuel,paint_color,car_type,price,sold_at
0,140411,100,01/02/2012,diesel,black,convertible,11300,01/01/2018
1,13929,317,01/04/2016,petrol,grey,convertible,69700,01/02/2018
2,183297,120,01/04/2012,diesel,white,convertible,10200,01/02/2018
3,128035,135,01/07/2014,diesel,red,convertible,25100,01/02/2018
4,97097,160,01/12/2014,diesel,silver,convertible,33400,01/04/2018


In [4]:
df.isnull().sum()

mileage              0
engine_power         0
registration_date    0
fuel                 0
paint_color          0
car_type             0
price                0
sold_at              0
dtype: int64

In [5]:
df["registration_year"]=df["registration_date"].str[6:].astype('int32')
df.head()

,mileage,engine_power,registration_date,fuel,paint_color,car_type,price,sold_at,registration_year
0,140411,100,01/02/2012,diesel,black,convertible,11300,01/01/2018,2012
1,13929,317,01/04/2016,petrol,grey,convertible,69700,01/02/2018,2016
2,183297,120,01/04/2012,diesel,white,convertible,10200,01/02/2018,2012
3,128035,135,01/07/2014,diesel,red,convertible,25100,01/02/2018,2014
4,97097,160,01/12/2014,diesel,silver,convertible,33400,01/04/2018,2014


In [6]:
df=df.drop(['registration_date'],axis='columns')
df.head()

,mileage,engine_power,fuel,paint_color,car_type,price,sold_at,registration_year
0,140411,100,diesel,black,convertible,11300,01/01/2018,2012
1,13929,317,petrol,grey,convertible,69700,01/02/2018,2016
2,183297,120,diesel,white,convertible,10200,01/02/2018,2012
3,128035,135,diesel,red,convertible,25100,01/02/2018,2014
4,97097,160,diesel,silver,convertible,33400,01/04/2018,2014


In [7]:
df=df.drop(['sold_at'],axis='columns')
df.head()

,mileage,engine_power,fuel,paint_color,car_type,price,registration_year
0,140411,100,diesel,black,convertible,11300,2012
1,13929,317,petrol,grey,convertible,69700,2016
2,183297,120,diesel,white,convertible,10200,2012
3,128035,135,diesel,red,convertible,25100,2014
4,97097,160,diesel,silver,convertible,33400,2014


In [8]:
df.head()
df.to_csv('bmw_cleaned.csv',index=False)

In [9]:
def AddDummiesForEncoding(columnArr,df):
    for col in columnArr:
        dummies = pd.get_dummies(df.fuel)
        df = pd.concat([df,dummies],axis="columns")
        
    return df

df1=AddDummiesForEncoding(['fuel','paint_color','car_type'],df)
df1.head()

,mileage,engine_power,fuel,paint_color,car_type,price,registration_year,diesel,electro,hybrid_petrol,petrol,diesel,electro,hybrid_petrol,petrol,diesel,electro,hybrid_petrol,petrol
0,140411,100,diesel,black,convertible,11300,2012,1,0,0,0,1,0,0,0,1,0,0,0
1,13929,317,petrol,grey,convertible,69700,2016,0,0,0,1,0,0,0,1,0,0,0,1
2,183297,120,diesel,white,convertible,10200,2012,1,0,0,0,1,0,0,0,1,0,0,0
3,128035,135,diesel,red,convertible,25100,2014,1,0,0,0,1,0,0,0,1,0,0,0
4,97097,160,diesel,silver,convertible,33400,2014,1,0,0,0,1,0,0,0,1,0,0,0


In [10]:
df2 = df1.drop(['fuel','paint_color','car_type'],axis='columns')
df2.head()

,mileage,engine_power,price,registration_year,diesel,electro,hybrid_petrol,petrol,diesel,electro,hybrid_petrol,petrol,diesel,electro,hybrid_petrol,petrol
0,140411,100,11300,2012,1,0,0,0,1,0,0,0,1,0,0,0
1,13929,317,69700,2016,0,0,0,1,0,0,0,1,0,0,0,1
2,183297,120,10200,2012,1,0,0,0,1,0,0,0,1,0,0,0
3,128035,135,25100,2014,1,0,0,0,1,0,0,0,1,0,0,0
4,97097,160,33400,2014,1,0,0,0,1,0,0,0,1,0,0,0


In [17]:
x = df2.drop(['price'],axis='columns').values
y = df2.price
x_train,x_test,y_train,y_test= sklearn.model_selection.train_test_split(x,y,test_size=0.28)

In [18]:
parameters_grid = {
    'criterion' : ['mse','friedman_mse'],  
         'splitter': ['best','random'],
            "max_depth" : [1,3,5]
}
model_1 = sklearn.model_selection.GridSearchCV(sklearn.tree.DecisionTreeRegressor(), 
                                               parameters_grid,  cv=10)
model_1.fit(x_train, y_train)
print("Accuracy of best decision tree classfier = {:.2f}".format(model_1.best_score_))
print("Best found hyperparameters of decision tree classfier = {}".format(model_1.best_params_))

Accuracy of best decision tree classfier = 0.66
Best found hyperparameters of decision tree classfier = {'criterion': 'mse', 'max_depth': 5, 'splitter': 'best'}


In [19]:
parameters_grid = {
    'normalize': [True, False]
}
model_1 = sklearn.model_selection.GridSearchCV(LinearRegression(), 
                                               parameters_grid,  cv=10)
model_1.fit(x_train, y_train)
print("Accuracy of best LinearRegression = {:.2f}".format(model_1.best_score_))
print("Best found hyperparameters of LinearRegression = {}".format(model_1.best_params_))

Accuracy of best LinearRegression = 0.64
Best found hyperparameters of LinearRegression = {'normalize': True}


In [21]:
parameters_grid = {
    'alpha': [1,2],
    'selection': ['random', 'cyclic']
}
model_1 = sklearn.model_selection.GridSearchCV(Lasso(), 
                                               parameters_grid,  cv=10)
model_1.fit(x_train, y_train)
print("Accuracy of best Lasso = {:.2f}".format(model_1.best_score_))
print("Best found hyperparameters of Lasso = {}".format(model_1.best_params_))

Accuracy of best Lasso = 0.64
Best found hyperparameters of Lasso = {'alpha': 2, 'selection': 'random'}


In [22]:
parameters_grid = {
    'criterion' : ['mse'],  
         'splitter': ['best'],
            "max_depth" : [5]
}
dcTree_clf_withBestHyperparameters = sklearn.tree.DecisionTreeRegressor( criterion ='mse',splitter= 'best' ,  max_depth = 5 )
dcTree_clf_withBestHyperparameters.fit(x_train,y_train)
y_pred=dcTree_clf_withBestHyperparameters.predict(x_test)
predicted_df=pd.DataFrame(y_pred,columns=['PredictedPrice'])
predicted_df.head()

,PredictedPrice
0,14657.978196
1,14657.978196
2,11087.827225
3,24702.173913
4,13625.268817


In [23]:
r2_score(y_test, y_pred)

0.5599675986319532